In [1]:
import json
import re
from collections import defaultdict
import pandas as pd
import pprint

## Loading data

Reading dataset information 

In [2]:
with open("od_edm_dataset_metadata.json") as f:
    metadata_datasets = json.load(f)
print("# of datasets:",len(metadata_datasets))

# of datasets: 584


Reading manually clasified columns

In [3]:
with open("ground.json") as f:
    ground = json.load(f)
with open("ground_by_cat.json") as f:
    ground_by_cat = json.load(f)
with open("ground_by_concept.json") as f:
    ground_by_concept = json.load(f)
print("# of columns labeled:",len(ground))
print("# of categories:",len(ground_by_cat))
print("# of concepts:",len(ground_by_concept))

# of columns labeled: 504
# of categories: 26
# of concepts: 12


In [4]:
ground_by_cat.keys()

dict_keys(['date', 'year', 'type', 'NaN', 'address', 'latitude', 'longitude', 'location', 'postal_code', 'ward_name', 'neighbourbood_id', 'neighbourbood_name', 'population', 'time', 'url', 'month_number', 'gender', 'age', 'language', 'quadrant', 'ward_number', 'phone', 'email', 'district', 'day', 'month_name'])

In [5]:
cat_type_number = ['year','latitude','longitude','neighbourbood_id','population','month_number','age','ward_number']
cat_type_date = ['date']
cat_type_location = ['location']

def type_of(cat):
    if cat in cat_type_number:
        return 'number'
    elif cat in cat_type_date:
        return 'date'
    elif cat in cat_type_location:
        return 'location'
    else:
        return 'text'

## Simillarity analysis
Creating column information excluding datasets tagged as _survey_

In [6]:
column_info = []
n = 0
num_words = 4
for ds in metadata_datasets:
    if 'survey' not in ds['tags']:
        n+=1
        for col in ds['columns']:
            cname = col['name'].strip().replace('_',' ')
            if not re.match(r".+[0-9]+",cname):
                if len(cname.split(' '))<=num_words:
                    col_info = {'name':cname, 'type':col['type'],'desc':col['description'], 'dataset':ds['id']}
                    column_info.append(col_info)
print("# of dataset to use:",n)
print("# of possible columns: ",len(column_info))

# of dataset to use: 506
# of possible columns:  3589


Processing using word embedding

In [7]:
import spacy
from spacy import displacy
# Load English tokenizer, tagger, parser, NER and word vectors
# nlp = spacy.load('en')
# nlp = spacy.load('en_core_web_md')
nlp = spacy.load('en_core_web_lg')
# nlp = spacy.load('en_vectors_web_lg')

In [8]:
n_of_words = 4
column_name_type = {}
n = 0
for ds in metadata_datasets:
    if 'survey' not in ds['tags']:
        n+=1
        for col in ds['columns']:
            cname = col['name'].strip().replace('_',' ')
            if not re.match(r".+[0-9]+",cname):
                if len(cname.split(' '))<=n_of_words:
                    key = (cname,col['type'])
                    column_name_type.setdefault(key,{'count':0,'max_sim':0,'col_sim':None})
                    column_name_type[key]['count']+=1

In [9]:
def similarity_between_columns(column_name_type, threshold):
    for (cn,t),c in column_name_type.items():
        c['nlp'] = nlp("{} ({})".format(cn,t))

    for cname1,info1 in column_name_type.items():
        msi = [cname1,0]
        pattern = ".*{}".format(cname1[0][:3])
        if cname1[0][0] =='(':
            pattern = ".*{}".format(cname1[0][1:4])
        for cname2,info2 in column_name_type.items():
            if cname1 != cname2:
                if re.match(pattern,cname2[0]):
                    sim = info1['nlp'].similarity(info2['nlp'])
                    if sim<1.0 and sim > msi[1]:
                        msi = [cname2,sim]
        if msi[1] >= threshold and msi[1]<=1.0:
            info1['col_sim'] = msi
        else:
            info1['col_sim'] = None

In [10]:
similarity_between_columns(column_name_type, threshold=0.911)

Update column_info with similarity between columns

In [11]:
for col in column_info:
    cname = (col['name'],col['type'])
    col['col_sim'] = column_name_type[cname]['col_sim']

Getting vector embedding for each pair _(column name, type)_ and embedding for column description

In [12]:
for col in column_info:
    col['nlp'] = column_name_type[(col['name'],col['type'])]['nlp']
    col['nlp_desc'] = nlp(col['desc'])

Computes the similarity between each column (name,type) and the categories

In [13]:
def similarity_between_column_category(column_info, categories, threshold, k=10):
    neighb = []
    neighb_sim = []
    cat_nlp = {cat:nlp("{} ({})".format(cat.replace('_',' '),type_of(cat))) for cat in categories}
    for col in column_info:
        nlp1 = col['nlp']
        msim = [None,0]
        for cat in categories:
            nlp2 = cat_nlp[cat]
            sim = nlp1.similarity(nlp2)
            if sim<=1.0 and sim >= threshold and sim>msim[1]:
                msim = [cat,sim]
        col['cat_sim'] = msim

In [14]:
similarity_between_column_category(column_info,ground_by_cat.keys(), threshold=0.95)

Using WordNet to obtain definitions from categories and concepts

In [16]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /home/abuss/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
cat_def = []
for cat in ground_by_cat.keys():
    if cat!='NaN':
        syns = wordnet.synsets(cat.split('_')[0])
        for s in syns:
            if s.pos()=='n':
                name = s.name().split('.')[0]
                cat_def.append((cat,s))

Similarity between category definitions (from Wordnet) and the column's description

In [18]:
def neighbourhood_desc_similarity(cat_def, column_info, threshold,k=10):
    neighb = []
    neighb_sim = []
    
    for col2 in column_info:
        col2['desc_sim'] = None
        if len(col2['desc'])>0: # If column has description
            d2 = col2['nlp_desc']
            maxsim = 0
            maxsyn = None
            for desc in cat_def:
                n1 = desc[1].name()
                d1 = nlp(desc[1].definition())
                sim = d1.similarity(d2)
                if sim >= threshold and sim>maxsim:
                    maxsim = sim
                    maxsyn = desc
            if maxsyn:
                col2['desc_sim'] = (maxsyn[0],maxsyn[1],maxsim)

In [19]:
neighbourhood_desc_similarity(cat_def,column_info,0.5)

In [20]:
tmp = pd.DataFrame(column_info)

In [21]:
tmp.count()

cat_sim     3589
col_sim     1987
dataset     3589
desc        3589
desc_sim    2364
name        3589
nlp         3589
nlp_desc    3589
type        3589
dtype: int64

Concept similarity

In [22]:
concept_def = []
for conc in ground_by_concept.keys():
    syns = wordnet.synsets(conc)
    for s in syns:
        if s.pos()=='n':
            name = s.name().split('.')[0]
            concept_def.append((conc,s))

Similarity between concept definitions (from Wordnet) and the column's description

In [23]:
def concept_desc_similarity(concept_def, column_info, threshold, k=10):
    neighb = []
    neighb_sim = []
    
    for col2 in column_info:
        col2['concept_sim'] = None
        if len(col2['desc'])>0: # If column has description
            d2 = col2['nlp_desc']
            maxsim = 0
            maxsyn = None
            for condef in concept_def:
                n1 = condef[1].name()
                d1 = nlp(condef[1].definition())
                sim = d1.similarity(d2)
                if sim >= threshold and sim>maxsim:
                    maxsim = sim
                    maxsyn = condef
            if maxsyn:
                col2['concept_sim'] = (maxsyn[0],maxsyn[1],maxsim)

In [24]:
concept_desc_similarity(concept_def,column_info, threshold=0.5)

In [33]:
def check_similarity(category,concept,col_cat,col_cat_desc,col_concept):
    ok_cat_name=False
    ok_cat_desc=False
    ok_concept=False

    if category:
        # Category baed on similarity between category name
        if col_cat[0]==category:
            ok_cat_name=True
        # Category baed on similarity between category definition
        if col_cat_desc:
            if col_cat_desc[0]==category:
                ok_cat_desc=True
    # Concept
    if col_concept and col_concept[0]==concept:
        ok_concept=True
    return ok_cat_name,ok_cat_desc,ok_concept

In [39]:
def concept_based_on_category(cat):
    if cat in ['date', 'year', 'time', 'month_number', 'month_name','day']:
        return 'time'
    if cat in ['postal_code', 'ward_name', 'ward_number', 'neighbourhood_id', 'neighbourhood_name', 'quadrant','district']:
        return 'area'
    if cat in ['address', 'latitude', 'longitude', 'location']:
        return 'location'
    if cat in ['gender']:
        return 'sex'
    return cat

In [40]:
def select_better(col):
    col_sim = col['col_sim']
    cat_sim = col['cat_sim']
    desc_sim = col['desc_sim']
    concept_sim = col['concept_sim']
    
    factor = 1.015
    cat_sim_v = cat_sim[1] if cat_sim[0] else 0
    desc_sim_v = desc_sim[2] if desc_sim else 0
    concept_sim_v = concept_sim[2]*factor if concept_sim else 0
    
    if cat_sim_v == desc_sim_v == concept_sim_v == 0:
        return [None,0]

    if cat_sim_v > desc_sim_v:
        if cat_sim_v > concept_sim_v:
            return concept_based_on_category(cat_sim[0]),cat_sim[1]
        else:
            return concept_sim[0],concept_sim_v
    else:
        if desc_sim_v > concept_sim_v:
            return concept_based_on_category(desc_sim[0]),desc_sim_v
        else:
            return concept_sim[0],concept_sim_v    

In [41]:
def assign_concept_no_fw(column_info):
    no_forwards = 0
    for col in column_info:
        concept = select_better(col)
        col['concept'] = concept
    return no_forwards

In [51]:
def assign_concept(column_info):
    column_info_dict_name_type = {}
    for col in column_info:
        key = (col['name'],col['type'])
        column_info_dict_name_type.setdefault(key,[])
        column_info_dict_name_type[key].append(col)

    no_forwards = 0
    for col in column_info:
        concept = select_better(col)
        col_sim = col['col_sim']
        if col_sim:
            forward = True
            key,s = col_sim
            possible = column_info_dict_name_type[key]
            i = 0
            found = False
            concept_f = [None,0]
            while i<len(possible) and not found:
                col2 = possible[i]
                i+=1
                concept_f = select_better(col2)
                if concept_f[0]:
                    found = True
        if concept[1]>concept_f[1]:
            col['concept'] = concept
        else:
            col['concept'] = concept_f
            no_forwards+=1
    return no_forwards

In [56]:
def validate(ground,column_info):
    column_info_dict = {}
    for col in column_info:
        key = (col['dataset'],col['name'])
        column_info_dict[key] = col

    ok_concept_count = 0
    no_ok_concept_count = 0
    failed = []
    report = []
    for gt in ground:
        key = (gt['dataset'],gt['id'])
        category = gt['category']
        if not pd.notna(category):
            category = None
        concept = gt['concept']

        col = column_info_dict[key]
        col_concept = col['concept']

        if concept==col_concept[0]:
            ok_concept_count += 1
            ok_concept = True
        else:
            no_ok_concept_count += 1
            ok_concept = False

        rep = {'name':gt['id'],'concept':concept,'col_concept':col_concept,'ok_concept':ok_concept,
    #            'identified':(ok_cat_name or ok_cat_desc or ok_concept),'forward':forward,
               'concept_sim':col_concept[1]
              }
        report.append(rep)

    report_df = pd.DataFrame(report)
    print("Ok:",ok_concept_count," no Ok:",no_ok_concept_count)
    return report_df

Validation __without__ forward

In [57]:
no_for = assign_concept_no_fw(column_info)

In [58]:
rep1 = validate(ground,column_info)

Ok: 315  no Ok: 189


Validation __with__ forward

In [59]:
no_for = assign_concept(column_info)
no_for

1701

In [60]:
rep2 = validate(ground,column_info)

Ok: 340  no Ok: 164


In [63]:
rep2

,col_concept,concept,concept_sim,name,ok_concept
0,"(time, 0.9473780807038001)",time,0.947378,responsedate,True
1,"(time, 0.9473780807038001)",time,0.947378,completiondate,True
2,"(language, 0.927227285402777)",time,0.927227,budget year,False
3,"(time, 0.9010908261475906)",type,0.901091,fund type,False
4,"(email, 0.730360308361521)",type,0.730360,category,False
5,"(neighbourbood_name, 0.9705253497834311)",name,0.970525,name,False
6,"(location, 1.0)",location,1.000000,address,True
7,"(location, 1.0)",location,1.000000,latitude,True
8,"(location, 1.0)",location,1.000000,longitude,True
9,"(location, 1.0)",location,1.000000,location,True
